## RNN: Sentiment Analysis

Get the data from here http://mng.bz/0tIo 

place the file in the `datasets` directory and unzip it 

`wget --no-check-certificate http://mng.bz/0tIo`

`unzip 0tIo`

Let's prepare the experiment

In [1]:
import os
imdb_dir = 'datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [2]:
print(len(texts))
print(texts[0])

25000
Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.


## Preparing the Data

When working with text we need to tokenize the data. In addition, the sequences we will pass to the RNNs need to be of the same length. To that aim we cut them at a certain length and if they are not long enough, we pad them with 0s at the beginning or the end of the sequence.

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# maximum review length
maxlen = 100
validation_samples = 10000 
# maximum words in our vocabulary
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print(data[0])

Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  777   16   28    4    1  115 2278 6887   11   19
 1025    5   27    5   42 2425 1861  128 2270    5    3 6985  308    7
    7 3383 2373    1   19   36  463 3169    2  222    3 1016  174   20
   49  808]


let's reconstruct a sequence to check that we are doing the right job

In [4]:
words = []
for el in data[0]:
    w = [k for k,v in word_index.items() if v==el]
    if w:
        words.append(w[0])

print(" ".join(words))

working with one of the best shakespeare sources this film manages to be to it's source whilst still appealing to a wider audience br br branagh steals the film from under nose and there's a talented cast on good form


And as with any other ML problem, let's split the dataset

In [5]:
from sklearn.model_selection import train_test_split

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.3)

## Load Embeddings

I hope we are familiar with word vectors, and the concept of embeddings...

Go [here](https://nlp.stanford.edu/projects/glove/) and download the Glove vectors. Place the file in your `dataset` directory and unzip it

Word Vectors are numerical representations of words that encode semantic meaning. Such encoding is attained by examining the context of a given word. In other words, words surrounding by similar words will have similar encodings/embeddings.

In [6]:
glove_dir = 'datasets/glove.6B'
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [7]:
embeddings_index['the']

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

## Build Embedding Matrix

In [8]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
                    embedding_matrix[i] = embedding_vector

## Model1: non trainable embeddings

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Conv1D, MaxPooling1D

model = Sequential()
model.add(Embedding(max_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=maxlen,
                    trainable=False))
model.add(LSTM(embedding_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
# or 
# model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 80,501
Non-trainable params: 1,000,000
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 17500 samples, validate on 7500 samples
Epoch 1/5
17500/17500 [==============================] - 73s 4ms/step - loss: 0.6170 - acc: 0.6485 - val_loss: 0.4814 - val_acc: 0.7809
Epoch 2/5
17500/17500 [==============================] - 70s 4ms/step - loss: 0.4730 - acc: 0.7746 - val_loss: 0.4146 - val_acc: 0.8105
Epoch 3/5
17500/17500 [==============================] - 71s 4ms/step - loss: 0.4303 - acc: 0.8009 - val_loss: 0.3951 - val_acc: 0.8221
Epoch 4/5
17500/17500 [==============================] - 71s 4ms/step - loss: 0.3991 - acc: 0.8195 - val_loss: 0.3717 - val_acc: 0.8387
Epoch 5/5
17500/17500 [==============================] - 71s 4ms/step - loss: 0.3777 - acc: 0.8270 - val_loss: 0.3571 - val_acc: 0.8455


## Model2: trainable embeddings

In [11]:
model2 = Sequential()
model2.add(Embedding(max_words,
                    embedding_dim,
                    input_length=maxlen))
model2.add(LSTM(embedding_dim, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.layers[0].set_weights([embedding_matrix])

In [12]:
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model2.fit(x_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 17500 samples, validate on 7500 samples
Epoch 1/5
17500/17500 [==============================] - 81s 5ms/step - loss: 0.5318 - acc: 0.7251 - val_loss: 0.3527 - val_acc: 0.8460
Epoch 2/5
17500/17500 [==============================] - 78s 4ms/step - loss: 0.3324 - acc: 0.8562 - val_loss: 0.3198 - val_acc: 0.8601
Epoch 3/5
17500/17500 [==============================] - 79s 5ms/step - loss: 0.2584 - acc: 0.8963 - val_loss: 0.3181 - val_acc: 0.8628
Epoch 4/5
17500/17500 [==============================] - 82s 5ms/step - loss: 0.2072 - acc: 0.9201 - val_loss: 0.3558 - val_acc: 0.8577
Epoch 5/5
17500/17500 [==============================] - 79s 5ms/step - loss: 0.1625 - acc: 0.9397 - val_loss: 0.3529 - val_acc: 0.8617


## Model3: RNNs and CNNs for text classification

In [13]:
model3 = Sequential()
model3.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model3.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model3.add(Dense(1, activation='sigmoid'))
model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = False
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,062,933
Trainable params: 62,933
Non-trainable params: 1,000,000
_________________________________________________________________
None


In [14]:
model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model3.fit(x_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 17500 samples, validate on 7500 samples
Epoch 1/5
17500/17500 [==============================] - 42s 2ms/step - loss: 0.5725 - acc: 0.6908 - val_loss: 0.4390 - val_acc: 0.7979
Epoch 2/5
17500/17500 [==============================] - 41s 2ms/step - loss: 0.4652 - acc: 0.7817 - val_loss: 0.4551 - val_acc: 0.8003
Epoch 3/5
17500/17500 [==============================] - 41s 2ms/step - loss: 0.4274 - acc: 0.8041 - val_loss: 0.3806 - val_acc: 0.8300
Epoch 4/5
17500/17500 [==============================] - 41s 2ms/step - loss: 0.4051 - acc: 0.8158 - val_loss: 0.3843 - val_acc: 0.8304
Epoch 5/5
17500/17500 [==============================] - 39s 2ms/step - loss: 0.3818 - acc: 0.8265 - val_loss: 0.3710 - val_acc: 0.8357
